# Transformer学习笔记

论文《Attention Is All You Need》提出了一个全新的完全基于Attention机制的Transformer模型，该模型不需要CNN或者RNN，性能突出，训练更快。

论文地址： [Attention Is All You Need](https://arxiv.org/pdf/1706.03762.pdf)

GitHub已经有人实现了Transformer模型，比如： [Kyubyong/transformer](https://github.com/Kyubyong/transformer/blob/master/modules.py)

当然，Google自己也有实现，见[tensorflow/tensor2tensor](https://github.com/tensorflow/tensor2tensor)

我是根据Kyubyong的实现来理解Transformer模型的。

根据论文，Transformer模型的架构如下图：

![transformer architecture](images/transformer.jpg)

这张图的结构很清晰，Kyubyong的实现也是完全根据这张图的结构一步一步来的。那我们逐步就理解一下代码。

## Normalize
代码如下：

In [ ]:
import tensorflow as tf

def normalize(inputs, epsilon=1e-8, scope="normalize", reuse=None):
    with tf.variable_scope(scope, reuse=reuse):
        # 获取inputs的形状
        inputs_shape = inputs.get_shape()
        # 参数的形状，是inputs形状的最后一行
        # 这种情况有点类似于 f= xW + b 中，把b合并到W和x中，x最后一行加上全是1的向量，W中最后一列增加一列
        params_shape = inputs_shape[-1:]
        
        # 求出inputs的均值和方差，两个张量的维度和inputs保持一致
        mean, variance = tf.nn.moments(inputs, [-1], keep_dims=True)
        
        beta = tf.Variable(tf.zeros(params_shape))
        gamma = tf.Variable(tf.ones(params_shape))
        # 使用epsilon是为了防止数值计算错误
        normalized = (inputs - mean) / ((variance + epsilon) ** (.5))
        
        # 经过线性计算，得到输出output
        outputs = gamma * normalized + beta
    
    return output

## Embedding
词嵌入阶段，这个大家应该很熟悉了。代码如下：

In [ ]:
import tensorflow as tf

def embedding(inputs, vocab_size, num_units, zero_pad=True, scale=True, scope="embedding", reuse=None):
    """词嵌入。
    
    Args:
        inputs: 输入张量
        vocab_size: 词典数量
        num_units: 隐藏层单元数量，也就是常说的embedding_size
        zero_pad: 是否增加一行全0的向量
    """
    
    with tf.variable_scope(scope, reuse=reuse):
        lookup_table = tf.get_variable("lookup_table",
                                      dtype=tf.float32,
                                      shape=[vocab_size, num_units],
                                      initializer=tf.contrib.layers.xavier_initializer())
        if zero_pad:
            # 在lookup_table的第一行加上一行全是0的向量
            lookup_table = tf.concat((tf.zeros(shape=[1, num_units]),
                                      lookup_table[1:,:]),0)
        
        output = tf.nn.embedding_lookup(lookup_table, inputs)
        if scale:
            output = output * (num_units ** 0.5)
        
    return output

## Positional Encoding
什么是Positional encoding? 我一开始接触这个词的时候，一脸懵逼。因为传统的seq2seq好像没有这个概念，即使是带有Attention机制的seq2seq也没有这个概念。

我们看懂代码，再来回答这个问题。

In [ ]:
import tensorflow as tf
import numpy as np

def positional_encoding(inputs, num_units, zero_pad=True, scale=True, scope="positional_encoding", reuse=None):
    """正弦位置编码。
    
    Args:
        inputs: 输入，2维张量
        num_units: 输出维度
        zero_pad: 是否增加一行全0的向量
        scale: 是否乘以num_units的平方根
    """
    # N就是batch_size，T就是time_steps
    N, T = inputs.get_shape().as_list()
    with tf.variable_scope(scope, reuse=reuse):
        position_ind = tf.tile(tf.expand_dims(tf.range(T), 0), [N, 1])

        # 根据论文的公式，得到一个二维张量
        position_enc = np.array([
            [pos / np.power(10000, 2.*i/num_units) for i in range(num_units)]
            for pos in range(T)])

        # 奇数列使用cosine计算，偶数列使用sin计算
        position_enc[:, 0::2] = np.sin(position_enc[:, 0::2])  # dim 2i
        position_enc[:, 1::2] = np.cos(position_enc[:, 1::2])  # dim 2i+1

        # 转化成一个张量，通过position就可以查出对应的数值，这一点和embedding十分相似
        lookup_table = tf.convert_to_tensor(position_enc)

        if zero_pad:
            # 在第一行加上一行全0的向量
            lookup_table = tf.concat((tf.zeros(shape=[1, num_units]),
                                      lookup_table[1:, :]), 0)
        # 根据位置，查表，获取输出
        outputs = tf.nn.embedding_lookup(lookup_table, position_ind)

        if scale:
            outputs = outputs * num_units**0.5

    return outputs